In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mne
import config_chbp_eeg as cfg


In [ ]:
# some viz config
small_size, medium_size, bigger_size = 13, 18, 18

plt.rc('font', size=small_size)          # controls default text sizes
plt.rc('axes', titlesize=small_size)     # fontsize of the axes title
plt.rc('axes', labelsize=medium_size)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('ytick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('legend', fontsize=small_size)    # legend fontsize
plt.rc('figure', titlesize=bigger_size)  # fontsize of the figure title

## First we read in some meta data needed for the present task

1. we need the demographic information

2. we need to know for which participants we have data

3. we need to sync both pieces of information

In [ ]:
from sklearn.model_selection import train_test_split


df_demographics = pd.read_csv('./inputs/Demographic_data.csv', header=1)

# remove empty columns
df_demographics = df_demographics.iloc[:, :5].set_index('Code')
df_demographics


# The later code uses the prefix "sub-" in the participants identifier, we will add it here to be fine

df_demographics.index = "sub-" + df_demographics.index
df_demographics.index

In [ ]:
# now we read in the processing log to see for which participants we have EEG

proc_log = pd.read_csv('./outputs/autoreject_log.csv')
good_subjects = proc_log.query('ok == "OK"').subject
good_subjects

In [ ]:
# then we filter the demographic list accordingly and establish the same order

df_demographics = df_demographics.loc[good_subjects]

In [ ]:
# Now we can put some data aside for testing and focus on 80 percent of the cases for exploring

train_cases, test_cases = train_test_split(df_demographics, test_size=.20, random_state=42)

In [ ]:
# let's plot the age distribution to get an idea who we are dealing with here

train_cases.Age.hist()
plt.ylabel('count')
plt.xlabel('Age [years]')

## Let's read in the pre-computed featurs

We first start with the power spectra. As we read, we make sure features are stored in
the same order as our meta info.

In [ ]:
features = mne.externals.h5io.read_hdf5('./outputs/features_eyes-closed.h5')
psds = [features[sub]['psds'] for sub in train_cases.index]
X_psd = np.array(psds)
print(X_psd.shape)  # hooray we have an array! :D

In [ ]:
# But what are these dimensions?

In [ ]:
# For better plotting options we may later need the channel layout
# As we picked common channels, we need to know what are the remaining channels.

ch_names = pd.read_csv('./outputs/common_channels_validated.csv').name.tolist()
print(ch_names)

In [ ]:
# We can get the frequency information from the features file

freqs = features[train_cases.index[0]]['meta_info']['freqs']

## Let's explore link between EEG power and age

In [ ]:
# We do something crude first: averaging over all channels and taking the log

X_psd_ave = np.log10(X_psd.mean(1))
X_psd_ave.shape

In [ ]:
# Let's visually explore the link between power and age by somple plotting with color codes
from sklearn.preprocessing import MinMaxScaler

y = train_cases.Age.values

age_sorter = np.argsort(y)
sorted_age = y[age_sorter]

ms = MinMaxScaler()
color_code = ms.fit_transform(sorted_age[:, None])[:, 0]
color = plt.matplotlib.cm.viridis(color_code)

plt.figure(figsize=(9, 7))
for ii, psd in enumerate(X_psd_ave[age_sorter]):
    plt.plot(freqs, psd, color=color[ii]);
plt.ylim(-16, -8)
plt.ylabel(r'$10 \times \log_{10}(PSD)$')
plt.xlabel("Frbequency [Hz]")
plt.show()

### Oh no!! We have a really bad outlier here! Lets' get rid of this one and do it again

In [ ]:
bad_idx = X_psd.mean(1).mean(1).argmin()
good_idx = list(range(len(X_psd)))
good_idx.pop(bad_idx)

y = train_cases.Age.values[good_idx]
age_sorter = np.argsort(y)
sorted_age = y[age_sorter]

X_psd = X_psd[good_idx]

ms = MinMaxScaler()
color_code = ms.fit_transform(sorted_age[:, None])[:, 0]
color = plt.matplotlib.cm.viridis(color_code)

plt.figure(figsize=(9, 7))
for ii, psd in enumerate(X_psd_ave[age_sorter]):
    plt.plot(freqs, psd, color=color[ii]);
plt.ylim(-16, -8)
plt.ylabel(r'$10 \times \log_{10}(PSD)$')
plt.xlabel("Frbequency [Hz]")
plt.show()

This does not reveal a clear pattern yet. Let's see if binning by age group gives a clearer picture. For this we shall add a gorup variable to our data frame. After looking at the histogram above, my gut feeling is telling me that we can do around 4 groups.

Now we als have to remember to remove that bad index from the data

In [ ]:
train_cases = train_cases.iloc[good_idx]

In [ ]:
train_cases['age_group'] = pd.cut(train_cases.Age, 4)

In [ ]:
# now we can do a group by after setting the index to range
train_cases = train_cases.reset_index()

In [ ]:
# note that we have now a 0-based RangeIndex that we can use to access the features
train_cases

In [ ]:
color = plt.cm.viridis(np.linspace(0.1, 0.9, 4))
plt.figure(figsize=(9, 7))
for ii, (key, inds) in enumerate(train_cases.groupby('age_group').groups.items()):
    plt.plot(freqs, X_psd_ave[inds].mean(0), color=color[ii], label=key, linewidth=3)
plt.legend()
plt.ylabel(r'$10 \times \log_{10}(PSD)$')
plt.xlabel("Frequency [Hz]")

Now we can see that that oldest people have much lower alpha power, which I would expect based on my prior knowledge.

## Spatial patterns

Yet another way to look at things is spatial. We now use the previously defined age groups to extract EEG topographies.
We will need MNE Python here to get this job done.

In [ ]:
# we compose a header object to handel channel locations
info = mne.create_info(ch_names=ch_names, ch_types=['eeg'] * len(ch_names), sfreq=200.0)

In [ ]:
# average EEG power for every channel across frequencies within age groups
age_groups = train_cases.groupby('age_group').groups

X_ave = np.array([X_psd[inds].mean(0).mean(1) for inds in age_groups.values()])

In [ ]:
X_ave.shape  # frequency bands, channels

In [ ]:
# fake being an MNE Evoked object :D
average_psds = mne.EvokedArray(data=X_ave.T, info=info)

In [ ]:
# now we need to add the channel information using a standard montage

montage = mne.channels.make_standard_montage('standard_1005')
average_psds.set_montage(montage)

In [ ]:
fig_psd = average_psds.plot_topomap(cmap='viridis', cbar_fmt='', vmin=np.min, vmax=np.max, show=False);
titles = [k for k in age_groups.keys()] + [r'$10 \times \log_{10}(PSD)$']
axes = fig_psd.findobj(plt.Axes)
for ii, ax in enumerate(axes):
    ax.set_title(titles[ii])
plt.show()

This is unfortunately inconclive. It is possible that occular artifacts are overshadowing what we're interested in. Let's do something crude and remove the frontal channels. But how?

In [ ]:
average_psds.plot_sensors(show_names=True);

In [ ]:
average_psds.drop_channels(['Fp1', 'Fp2'])

Will it be better now?

In [ ]:
fig_psd = average_psds.plot_topomap(cmap='viridis', cbar_fmt='', vmin=np.min, vmax=np.max, show=False);
titles = [k for k in age_groups.keys()] + [r'$10 \times \log_{10}(PSD)$']
axes = fig_psd.findobj(plt.Axes)
for ii, ax in enumerate(axes):
    ax.set_title(titles[ii])
plt.show()

Well let's say a bit.

In [ ]:
average_psds.drop_channels(['AF3', 'AF4'])

In [ ]:
fig_psd = average_psds.plot_topomap(cmap='viridis', cbar_fmt='', vmin=np.min, vmax=np.max, show=False);
titles = [k for k in age_groups.keys()] + [r'$10 \times \log_{10}(PSD)$']
axes = fig_psd.findobj(plt.Axes)
for ii, ax in enumerate(axes):
    ax.set_title(titles[ii])
plt.show()

# Additional exercise for plotting:



1. Compute pearson correlations (or linear models) to capture and visulaize the link between age and power at each frequency.
Then plot it!

2. how do things look for the eyes opened condition?

# A first naive model

Shall we still try to see if a stats model can do better than us and find patterns that predict the age?


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
freq_selection_idx = (freqs > 2) & (freqs < 20)

# let's select some channel regions and average

chan_selection = [ch for ch in ch_names if ch.startswith("P")]
chan_selection_idx = [info['ch_names'].index(ch) for ch in chan_selection]
X_parietal = X_psd[:, chan_selection_idx][:, :, freq_selection_idx]

chan_selection = [ch for ch in ch_names if ch.startswith("C")]
chan_selection_idx = [info['ch_names'].index(ch) for ch in chan_selection]
X_central = X_psd[:, chan_selection_idx][:, :, freq_selection_idx]

X_ = np.concatenate([X_central.mean(1), X_parietal.mean(1)], axis=1)

In [ ]:
X_.shape

In [ ]:
ridge = make_pipeline(StandardScaler(), Ridge(alpha=1))
y_pred = cross_val_predict(estimator=ridge, X=X_, y=y)
plt.scatter(y, y_pred)

In [ ]:
from sklearn.linear_model import GammaRegressor
gamma = make_pipeline(StandardScaler(), GammaRegressor(alpha=1))
y_pred_gamma = cross_val_predict(estimator=gamma, X=X_, y=y)
plt.scatter(y, y_pred_gamma)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_true=y, y_pred=y_pred_gamma))

In [ ]:
from IPython.display import Image
Image(filename='./outputs/demo.png')

# Additional exercise for modeling:


1. What happens if we do not remove the bad case?

2. Do these models generalize to the 50 left-out subjects?

3. Can performance be improved by removing the frontal channels? and would that be a good idea? 

4. Can you come up with better ad-hoc features based on what we have seen?